In [31]:
import re

from oauth2client.service_account import ServiceAccountCredentials
import gspread
import string


In [32]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds_sheet = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json', scope)
client = gspread.authorize(creds_sheet)

# Fetch PDF URLs from Google Sheets document
spreadsheet_key = '1xyQe3vgYKGl_hcPa1MOEn970xjTUirEUq_ewfiGjLRs'
sheet_name = 'Main'
worksheet = client.open_by_key(spreadsheet_key).worksheet(sheet_name)

In [33]:
texts = worksheet.col_values(167)
texts = [x for x in texts if x and x !='NOT_FOUND' and x !='error'] 
len(texts) 

166

In [36]:
texts

['drive_text',
 '4/24/23, 4:56 PM اﻟﻧﺷرة: ﻣظﺎھرة ﺿﺧﻣﺔ ﻓﻲ ﺑﻠدة اﻟﻔﺎﻛﮭﺔ ﻋﻠﻰ اﻟطرﯾﻖ اﻟدوﻟﯾﺔ اﻟﮭرﻣل ﺑﻌﻠﺑك .elnashra.com/news/show/1356594/ اﻟﻧﺷرة:-ﻣظﺎھرة-ﺿﺧﻣﺔ-ﺑﻠدة-اﻟﻔﺎﻛﮭﺔ-ﻋﻠﻰ-اﻟطرﯾﻖ-اﻟدوﻟﻲ 1/2 \ue906 اﻟﻨﺸﺮة - أﺧﺒﺎر ﻟﺒﻨﺎن واﻟﺸﺮق اﻷوﺳﻂ',
 '2/17/23, 2:14 PM اﻟوﻛﺎﻟﺔ اﻟوطﻧﯾﺔ ﻟﻺﻋﻼم - ﻣﺣﺗﺟون ﻗطﻌوا طرﯾﻖ اﻣﯾون وﻋﺎﺑﺎ .nna-leb.gov .lb/ar/pictures/31944/nna-leb.gov .lb 1/8ﻣﺤﺘﺠﻮن ﻗﻄﻌﻮا ﻃﺮﻳﻖ اﻣﻴﻮن وﻋﺎﺑﺎ 12:30 اﻟﺴﺎﻋﺔ 2019 ﺗﺸﺮﻳﻦ اﻷول 18 اﻟﺠﻤﻌﺔ \uf073 وﻃﻨﻴﺔ - أﻓﺎدت ﻣﻨﺪوﺑﺔ اﻟﻮﻛﺎﻟﺔ اﻟﻮﻃﻨﻴﺔ ﻟﻼﻋﻼم ﻣﻴﺸﺎﻻ ﺳﺎﺳﻴﻦ ان اﻟﻤﺤﺘﺠﻴﻦ ﻗﻄﻌﻮا ﻃﺮﻳﻖ ﻋﺎم ﺑﻠﺪة اﻣﻴﻮن ﻓﻲ اﻟﻜﻮرة ﺣﻴﺚ رﻛﻨﻮا ﺳﻴﺎراﺗﻬﻢ ﻓﻲ وﺳﻂ اﻟﻄﺮﻳﻖ راﻓﻌﻴﻦ اﻻﻋﻼم اﻟﻠﺒﻨﺎﻧﻴﺔ. ﻛﺬﻟﻚ ﺗﻢ ﻗﻄﻊ ﻃﺮﻳﻖ ﻣﻔﺮق ﺑﻠﺪة ﻋﺎﺑﺎ ﻣﻦ ﻗﺒﻞ ﺑﻌﺾ اﻟﻤﺤﺘﺠﻴﻦ أﻳﻀﺎ. =======(ar/7/ /) ﻣﺘﻔﺮﻗﺎت + − ﺗﻮﻳﺘﺮ (ar/6/ /) ﺳﻼم: رﺣﻢ اﻟﻠﻪ اﻻﺑﻄﺎل ﺷﻬﺪاء اﻟﻮﻃﻦ ar//595378/- /) -- -- ( أﻣﻞ أﺑﻮ زﻳﺪ: اﻟﻤﺆﺳﺴﺔ اﻟﻌﺴﻜﺮﻳﺔ ﻣﺎزاﻟﺖ ﺗﺪﻓﻊ اﻷﺛﻤﺎن اﻟﻐﺎﻟﻴﺔ ar//595375/-- /) -- - -- (- اﻟﺤﻮاط: ﻳﻤﻜﻨﻜﻢ اﻻﻧﺘﺤﺎر ﻟﻮﺣﺪﻛﻢ أﻣﺎ ﻧﺤﻦ ﻓﺴﻨﻮاﺟﻪ ﻣﺸﺮوﻋﻜﻢ اﻟﻤﺪﻣﺮ ar//595369/- /) - - ---FM 96.2 و98.1 و98.5 ﺗﺎﺑﻌﻮا أﺧﺒﺎر اﻟﻮﻛﺎﻟﺔ اﻟﻮﻃﻨﻴﺔ ﻟﻼﻋﻼم ﻋﺒﺮ أﺛﻴﺮ إذاﻋﺔ ﻟﺒﻨﺎن

In [34]:
def clean_text(text):
    new_text = []
    for x in text.split():
        x = ''.join(i for i in x if i not in string.digits and i not in string.ascii_letters and i not in string.punctuation)
        x = x.replace('�','')
        new_text.append(x)
    return ' '.join(x for x in new_text if x)

In [37]:
# Clean the text in the column
count = 0
cleaned_values = []
for text in texts:
    cell_text = worksheet.find(text, )
    cleaned_text = clean_text(text)  # Remove non-Arabic characters
    worksheet.update_cell(cell_text.row, 171, cleaned_text)
    count += 1
    print((count * 100)/len(texts))

0.6024096385542169
1.2048192771084338
1.8072289156626506
2.4096385542168677
3.0120481927710845
3.6144578313253013
4.216867469879518
4.819277108433735
5.421686746987952
6.024096385542169
6.626506024096385
7.228915662650603
7.831325301204819
8.433734939759036
9.036144578313253
9.63855421686747
10.240963855421686
10.843373493975903
11.44578313253012
12.048192771084338
12.650602409638553
13.25301204819277
13.855421686746988
14.457831325301205
15.060240963855422
15.662650602409638
16.265060240963855
16.867469879518072
17.46987951807229
18.072289156626507
18.674698795180724
19.27710843373494
19.879518072289155
20.481927710843372
21.08433734939759
21.686746987951807
22.289156626506024
22.89156626506024
23.49397590361446
24.096385542168676
24.698795180722893
25.301204819277107
25.903614457831324
26.50602409638554
27.10843373493976
27.710843373493976
28.313253012048193
28.91566265060241
29.518072289156628
30.120481927710845
30.72289156626506
31.325301204819276
31.927710843373493
32.530120481927